# What if California gave money to everyone, not only vehicle owners?

Last month, Governor Gavin Newsom [proposed an $11 billion package](https://www.gov.ca.gov/2022/03/23/governor-newsom-proposes-11-billion-relief-package-for-californians-facing-higher-gas-prices/) for Californians facing higher gas prices. Newsom reserved $9 billion of this to send each vehicle owner $400 for each registered vehicle, up to two per person.

In this report, we estimate the poverty impact of this policy and compare it against budget-neutral alternatives that include Californians who don't own vehicles.

In [2]:
from openfisca_us import Microsimulation, ACS
from ca_ubi.vehicle_payment import vehicle_subsidy
import numpy as np
from ca_ubi.ubi import create_ubi_reform

def set_non_ca_weights_to_zero(simulation: Microsimulation):
    person_weight = simulation.calc("person_weight").values
    simulation.set_input("person_weight", 2022, np.where(simulation.calc("state_code", map_to="person").values == "CA", person_weight, 0))
    household_weight = simulation.calc("household_weight").values
    simulation.set_input("household_weight", 2022, np.where(simulation.calc("state_code", map_to="household").values == "CA", household_weight, 0))
    spm_unit_weight = simulation.calc("spm_unit_weight").values
    simulation.set_input("spm_unit_weight", 2022, np.where(simulation.calc("spm_unit_fips").values == 6, spm_unit_weight, 0))
    return simulation

baseline = Microsimulation(dataset=ACS, year=2019)
baseline = set_non_ca_weights_to_zero(baseline)
vehicle_payment_simulation = Microsimulation(vehicle_subsidy, dataset=ACS, year=2019)
vehicle_payment_simulation = set_non_ca_weights_to_zero(vehicle_payment_simulation)
cost_of_vehicle_payment = vehicle_payment_simulation.calc("ca_vehicle_payment", map_to="household").sum()
ca_population = baseline.calc("people").sum()
ubi_amount = cost_of_vehicle_payment / ca_population
print(f"Vehicle payment costs ${cost_of_vehicle_payment / 1e9:.1f}bn, CA population == {ca_population / 1e6:.1f} million, UBI amount == ${ubi_amount:.0f}/year")
ubi_simulation = Microsimulation(create_ubi_reform(ubi_amount), dataset=ACS, year=2019)
ubi_simulation = set_non_ca_weights_to_zero(ubi_simulation)

/home/nikhil/policyengine/openfisca-us/openfisca_us/variables/demographic/person/vehicles_owned.py:28: RuntimeWarning: invalid value encountered in remainder
  % num_adults_in_household


Vehicle payment costs $8.3bn, CA population == 38.7 million, UBI amount == $215/year


In [3]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook_connected" # As per https://github.com/microsoft/vscode-jupyter/issues/4364#issuecomment-817352686
from ubicenter import format_fig
from ubicenter.plotly import BLUE_COLOR_SEQUENCE


def get_poverty_rates(simulation: Microsimulation, label: str, deep_poverty: bool = False) -> pd.DataFrame:
    in_poverty = simulation.calc(f"spm_unit_is_in{'_deep' if deep_poverty else ''}_spm_poverty", map_to="person")
    baseline_in_poverty = baseline.calc(f"spm_unit_is_in{'_deep' if deep_poverty else ''}_spm_poverty", map_to="person")
    group_names = ["Children", "Adults", "Seniors", "All"]
    poverty_rates = []
    for group in ("is_child", "is_wa_adult", "is_senior", "people"):
        poverty_rates.append(in_poverty[simulation.calc(group) > 0].mean() / baseline_in_poverty[baseline.calc(group) > 0].mean() - 1)
    return pd.DataFrame({
        "Group": group_names,
        "Poverty rate": poverty_rates,
        "Label": label,
        "Type": "Poverty" if not deep_poverty else "Deep poverty",
    })

def get_regular_and_deep_poverty_rates(simulation: Microsimulation, label: str) -> pd.DataFrame:
    regular_df = get_poverty_rates(simulation, label, deep_poverty=False)
    deep_df = get_poverty_rates(simulation, label, deep_poverty=True)
    return pd.concat([regular_df, deep_df])

def get_hovercards(poverty_rate_df: pd.DataFrame) -> list:
    hover_cards = []
    for i, row in poverty_rate_df.iterrows():
        poverty_label = "poverty" if row["Type"] == "Poverty" else "deep poverty"
        if row["Label"] == "Baseline":
            hover = f"The current {poverty_label} rate for <b>{row['Group'].lower()}</b> is <b>{row['Poverty rate']:.1%}</b>"
        else:
            baseline_rate = poverty_rate_df.loc[(poverty_rate_df["Label"] == "Baseline") & (poverty_rate_df["Type"] == row["Type"]), "Poverty rate"].iloc[0]
            hover = f"Under the {'vehicle payment' if row['Label'] == 'Vehicle payment' else 'UBI'}, the {poverty_label} rate for <b>{row['Group'].lower()}</b> would fall<br> by {(row['Poverty rate'] - baseline_rate) / baseline_rate:.1%}"    
        hover_cards.append(hover)
    return hover_cards

simulations = [baseline, vehicle_payment_simulation, ubi_simulation]
labels = ["Baseline", "Vehicle payment", "UBI"]
poverty_rate_df = pd.concat([get_regular_and_deep_poverty_rates(simulation, label) for simulation, label in zip(simulations, labels)])
poverty_rate_df["hover_card"] = get_hovercards(poverty_rate_df)

fig = px.bar(
    poverty_rate_df[(poverty_rate_df.Label != "Baseline") & (poverty_rate_df.Type == "Poverty")], 
    x="Group", 
    y="Poverty rate", 
    color="Label", 
    barmode="group", 
    color_discrete_sequence=BLUE_COLOR_SEQUENCE, 
    custom_data=["hover_card"],
).update_layout(
    title="Changes to poverty rates by age group",
    yaxis_tickformat=".0%",
    legend_title=""
).update_traces(
    hovertemplate="%{customdata[0]}",
)
format_fig(fig)

/home/nikhil/anaconda3/envs/policyengine/lib/python3.7/site-packages/ipykernel_launcher.py:36: RuntimeWarning:

divide by zero encountered in double_scalars



In [4]:
fig = px.bar(
    poverty_rate_df[(poverty_rate_df.Label != "Baseline") & (poverty_rate_df.Type == "Deep poverty")], 
    x="Group", 
    y="Poverty rate", 
    color="Label", 
    barmode="group", 
    color_discrete_sequence=BLUE_COLOR_SEQUENCE, 
    custom_data=["hover_card"],
).update_layout(
    title="Changes to deep poverty rates by age group",
    yaxis_tickformat=".0%",
    legend_title=""
).update_traces(
    hovertemplate="%{customdata[0]}",
)
format_fig(fig)

In [5]:
def get_ubi_performance(df: pd.DataFrame) -> pd.Series:
    # This dataframe will have three rows. We should subtract the row with Label "UBI" from the one with Label "Vehicle payment" and return the result
    return df.loc[(df.Label == "Vehicle payment"), "Poverty rate"] / df.loc[(df.Label == "UBI"), "Poverty rate"] - 1

grouped_df = poverty_rate_df.groupby(["Group", "Type"]).apply(get_ubi_performance).reset_index()
grouped_df["order"] = grouped_df["Group"].map({"Children": 0, "Adults": 1, "Seniors": 2, "All": 3})
grouped_df = grouped_df.sort_values(by=["order", "Type"])

fig = px.bar(
    grouped_df,
    x="Group",
    color="Type",
    y="Poverty rate",
    barmode="group",
    color_discrete_sequence=BLUE_COLOR_SEQUENCE,
).update_layout(
    title="Additional poverty changes with a UBI",
    yaxis_tickformat=".0%",
    yaxis_title="Change to poverty rate",
    legend_title="",
)
format_fig(fig)

In [12]:
def get_decile_df(baseline: Microsimulation, simulation: Microsimulation, label: str) -> pd.DataFrame:
    income = baseline.calc("spm_unit_net_income")
    people = simulation.calc("people", map_to="spm_unit")
    equiv_income = income / np.sqrt(people)
    new_income = simulation.calc("spm_unit_net_income")
    if label == "Vehicle payment":
        new_income += simulation.calc("ca_vehicle_payment")
    elif label == "UBI":
        new_income += simulation.calc("ca_basic_income", map_to="spm_unit")
    equiv_income.weights *= people.values
    income_decile = equiv_income.decile_rank()
    equiv_income.weights /= people.values
    gain = new_income.groupby(income_decile).sum() / income.groupby(income_decile).sum() - 1
    gain = gain[1:]
    return pd.DataFrame({
        "Decile": gain.index,
        "Change to net income": gain.values,
        "Reform": label,
    })

reform_simulations = [vehicle_payment_simulation, ubi_simulation]
labels = ["Vehicle payment", "UBI"]
decile_df = pd.concat([get_decile_df(baseline, simulation, label) for simulation, label in zip(reform_simulations, labels)])

fig = px.bar(
    decile_df, 
    y="Change to net income", 
    x="Decile", 
    color="Reform", 
    barmode="group", 
    color_discrete_sequence=BLUE_COLOR_SEQUENCE
).update_layout(
    title="Change to net income by decile",
    xaxis_title="Per-person income decile",
    yaxis_tickformat=".1$",
    xaxis_tickvals=list(range(1, 11)),
    legend_title="",
)
format_fig(fig)

In [7]:
df = pd.DataFrame({
    "State": baseline.calc("state_name"),
    "In poverty": baseline.calc("spm_unit_is_in_spm_poverty", map_to="household"),
    "In deep poverty": baseline.calc("spm_unit_is_in_deep_spm_poverty", map_to="household"),
    "Poverty threshold": baseline.calc("spm_unit_spm_threshold", map_to="household"),
    "Original net income": baseline.calc("spm_unit_net_income", map_to="household").round(),
    "Weight": baseline.calc("people", map_to="household").weights,
    "Adults": baseline.calc("is_adult", map_to="household"),
    "Children": baseline.calc("is_child", map_to="household"),
    "Vehicle subsidy": vehicle_subsidy_simulation.calc("ca_vehicle_subsidy_payment", map_to="household"),
    "UBI": ubi_simulation.calc("ca_basic_income", map_to="household"),
    "Vehicles owned": baseline.calc("household_vehicles_owned", map_to="household"),
    "Income-poverty line ratio": baseline.calc("spm_unit_net_income", map_to="household") / baseline.calc("spm_unit_spm_threshold", map_to="household"),
})

df[df.State == "CA"][df["In poverty"] > 0].corr()

NameError: name 'vehicle_subsidy_simulation' is not defined

In [ ]:
total_vehicles = baseline.calc("vehicles_owned").sum()/1e6
total_drivers = (baseline.calc("vehicles_owned") > 0).sum()/1e6

print(f"{total_vehicles:.1f} million vehicles, {total_drivers:.1f} million drivers.")

27.1 million vehicles, 21.6 million drivers.
